In [1]:
import pandas as pd
from connection import connect

In [10]:
#connection
SCHEMA='sales'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

['sales_person_quota_history', 'country_region_currency', 'sales_reason', 'credit_card', 'sales_tax_rate', 'currency', 'sales_territory', 'currency_rate', 'sales_territory_history', 'customer', 'shopping_cart_item', 'person_credit_card', 'special_offer', 'special_offer_product', 'sales_order_header_sales_reason', 'sales_order_detail', 'sales_order_header', 'sales_person', 'store']


## Extract

In [63]:
fact_internet_sales=pd.read_sql("""
SELECT  p.product_number, sh.order_date, sh.due_date, sh.ship_date,COALESCE(cr.to_currency_code, 'USD') AS to_currency_code
FROM sales.sales_order_header as sh
JOIN sales.sales_order_detail as so
ON sh.sales_order_id=so.sales_order_id
JOIN production.product as p
ON so.product_id=p.product_id
LEFT JOIN sales.currency_rate as cr
ON sh.currency_rate_id=cr.currency_rate_id
WHERE sh.online_order_flag=true
                                
 """,co_oltp)

t_special_offer = pd.read_sql("""
    SELECT p.product_number, sod.special_offer_id
    FROM sales.sales_order_detail AS sod
    JOIN production.product AS p
      ON sod.product_id = p.product_id
    JOIN sales.sales_order_header AS soh
      ON sod.sales_order_id = soh.sales_order_id
    WHERE soh.online_order_flag = true
""", co_oltp)



dim_product=pd.read_sql_table('dim_product',etl_conn)



dim_promotion=pd.read_sql_table('dim_promotion',etl_conn)



## Transform

In [64]:
#delete duplicates in t_special_offer and dim_product

dim_product=dim_product.drop_duplicates(subset=['product_alternate_key'])

t_special_offer.drop_duplicates(subset=['product_number'],inplace=True)

In [65]:
# join dim_product

fact_internet_sales=fact_internet_sales.merge(
    dim_product[['product_key','product_alternate_key']],
    left_on='product_number',
    right_on='product_alternate_key',
    how='left'
    
)

fact_internet_sales

,product_number,order_date,due_date,ship_date,to_currency_code,product_key,product_alternate_key
0,BK-R93R-62,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,CAD,1824,BK-R93R-62
1,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,FRF,1860,BK-M82S-44
2,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1860,BK-M82S-44
3,BK-R50B-62,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1850,BK-R50B-62
4,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,AUD,1860,BK-M82S-44
...,...,...,...,...,...,...,...
60393,FE-6654,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654
60394,CA-1098,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1737,CA-1098
60395,FE-6654,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654
60396,ST-1401,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,2000,ST-1401


In [66]:
#join whit dim_product

fact_internet_sales=fact_internet_sales.merge(
    t_special_offer,
    on='product_number',
    how='left'
).drop(['product_number'],axis=1)

fact_internet_sales

,order_date,due_date,ship_date,to_currency_code,product_key,product_alternate_key,special_offer_id
0,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,CAD,1824,BK-R93R-62,1
1,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,FRF,1860,BK-M82S-44,1
2,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1860,BK-M82S-44,1
3,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1850,BK-R50B-62,1
4,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,AUD,1860,BK-M82S-44,1
...,...,...,...,...,...,...,...
60393,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654,1
60394,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1737,CA-1098,1
60395,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654,1
60396,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,2000,ST-1401,1


In [67]:
#join whit dim_promotion

fact_internet_sales=fact_internet_sales.merge(
    dim_promotion[['promotion_alternate_key','promotion_key']],
    left_on='special_offer_id',
    right_on='promotion_alternate_key',
    how='left'
).drop(['special_offer_id'],axis=1)

fact_internet_sales



,order_date,due_date,ship_date,to_currency_code,product_key,product_alternate_key,promotion_alternate_key,promotion_key
0,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,CAD,1824,BK-R93R-62,1,1
1,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,FRF,1860,BK-M82S-44,1,1
2,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1860,BK-M82S-44,1,1
3,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,USD,1850,BK-R50B-62,1,1
4,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,AUD,1860,BK-M82S-44,1,1
...,...,...,...,...,...,...,...,...
60393,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654,1,1
60394,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1737,CA-1098,1,1
60395,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,1999,FE-6654,1,1
60396,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,USD,2000,ST-1401,1,1


In [68]:
# convert dates to integers

fact_internet_sales['order_date']=fact_internet_sales['order_date'].dt.strftime('%Y%m%d').astype(int)
fact_internet_sales['due_date']=fact_internet_sales['due_date'].dt.strftime('%Y%m%d').astype(int)
fact_internet_sales['ship_date']=fact_internet_sales['ship_date'].dt.strftime('%Y%m%d').astype(int)

In [69]:
#rename the columns to propper name in olap database

fact_internet_sales.rename(columns={'order_date':'order_date_key',
                                    'due_date':'due_date_key',
                                    'ship_date':'ship_date_key',    
                                    },inplace=True)

fact_internet_sales

,order_date_key,due_date_key,ship_date_key,to_currency_code,product_key,product_alternate_key,promotion_alternate_key,promotion_key
0,20110531,20110612,20110607,CAD,1824,BK-R93R-62,1,1
1,20110531,20110612,20110607,FRF,1860,BK-M82S-44,1,1
2,20110531,20110612,20110607,USD,1860,BK-M82S-44,1,1
3,20110531,20110612,20110607,USD,1850,BK-R50B-62,1,1
4,20110531,20110612,20110607,AUD,1860,BK-M82S-44,1,1
...,...,...,...,...,...,...,...,...
60393,20140630,20140712,20140707,USD,1999,FE-6654,1,1
60394,20140630,20140712,20140707,USD,1737,CA-1098,1,1
60395,20140630,20140712,20140707,USD,1999,FE-6654,1,1
60396,20140630,20140712,20140707,USD,2000,ST-1401,1,1


In [70]:
#join currency

dim_currency=pd.read_sql_table('dim_currency',etl_conn)

fact_internet_sales=fact_internet_sales.merge(
    dim_currency[['currency_key','currency_alternate_key']],
    left_on='to_currency_code',
    right_on='currency_alternate_key',
    how='left'
)

fact_internet_sales


,order_date_key,due_date_key,ship_date_key,to_currency_code,product_key,product_alternate_key,promotion_alternate_key,promotion_key,currency_key,currency_alternate_key
0,20110531,20110612,20110607,CAD,1824,BK-R93R-62,1,1,19,CAD
1,20110531,20110612,20110607,FRF,1860,BK-M82S-44,1,1,39,FRF
2,20110531,20110612,20110607,USD,1860,BK-M82S-44,1,1,100,USD
3,20110531,20110612,20110607,USD,1850,BK-R50B-62,1,1,100,USD
4,20110531,20110612,20110607,AUD,1860,BK-M82S-44,1,1,6,AUD
...,...,...,...,...,...,...,...,...,...,...
60393,20140630,20140712,20140707,USD,1999,FE-6654,1,1,100,USD
60394,20140630,20140712,20140707,USD,1737,CA-1098,1,1,100,USD
60395,20140630,20140712,20140707,USD,1999,FE-6654,1,1,100,USD
60396,20140630,20140712,20140707,USD,2000,ST-1401,1,1,100,USD
